\begin{equation}
\frac{\kappa_{\text{qubit}}}{\kappa_{\text{HO}}}
= \left( \frac{g_{\text{qubit}}}{g_{\text{HO}}} \right)^{2}.
\end{equation}



\begin{equation}
\boxed{
\frac{g_{\text{qubit}}}{g_{\text{HO}}}
= \frac{\langle 0_{\text{qubit}} | \hat{n} | 1_{\text{qubit}} \rangle}
       {\langle 0_{\text{HO}} | \hat{n} | 1_{\text{HO}} \rangle}
= \left( \frac{E_{L,\text{qubit}}}{E_{L,\text{HO}}} \right)^{1/4}
  \langle 0_{\text{qubit}} | \hat{a} - \hat{a}^\dagger | 1_{\text{qubit}} \rangle
 } 
\end{equation}


\begin{equation}
\boxed{
\kappa_{\text{qubit}}
= \kappa_{\text{HO}}
  \sqrt{\frac{E_{L,\text{qubit}}}{E_{L,\text{HO}}}}
  \left| \langle 0_{\text{qubit}} | \hat{a} - \hat{a}^\dagger | 1_{\text{qubit}} \rangle \right|^2
}
\end{equation}

\begin{equation}
\hat{n} \;=\; -\frac{i}{\sqrt{2}}
\left( \frac{E_L}{8E_C} \right)^{1/4}
\left( \hat{a} - \hat{a}^\dagger \right).
\end{equation}

\begin{equation}
\boxed{
\kappa_{\text{qubit}}
= \kappa_{\text{HO}}
  \sqrt{\frac{E_{L,\text{qubit}}}{E_{L,\text{HO}}}} \frac{\left| \langle 0_{\text{qubit}} | \hat{n} | 1_{\text{qubit}} \rangle \right|^2}{n_{zpf}}
  
}
\end{equation}


\begin{equation}
\kappa_{\text{qubit}}
= \kappa_{\text{HO}} * \left( \frac{g_{\text{qubit}}}{g_{\text{HO}}} \right)^{2}.
\end{equation}



In [4]:
import scqubits as scq

# ---------------------------
# Transmon example
# ---------------------------
EJ = 1.15   # Josephson energy in GHz
EC = 0.85    # Charging energy in GHz
ng = 0.0     # Offset charge
ncut = 30    # Hilbert space cutoff

transmon = scq.Transmon(EJ=EJ, EC=EC, ng=ng, ncut=ncut)

# 0->1 transition frequency
f01_transmon = transmon.eigenvals(evals_count=2)[1] - transmon.eigenvals(evals_count=2)[0]

# charge operator matrix element <0|n|1>
n_mat_elem_transmon = transmon.matrixelement_table('n_operator', evals_count=2)[0,1]

print("Transmon f01 = ", f01_transmon, "GHz")
print("<0|n|1> =", n_mat_elem_transmon)

# ---------------------------
# Fluxonium example
# ---------------------------
EL = 1.15   # Inductive energy in GHz
EC = 0.85     # Charging energy in GHz
EJ = 5    # Josephson energy in GHz
phi_ext = 0.5  # External flux in units of flux quantum
cutoff = 110   # size of Hilbert space

fluxonium = scq.Fluxonium(EL=EL, EC=EC, EJ=EJ, cutoff=cutoff, flux=phi_ext)

# 0->1 transition frequency
f01_fluxonium = fluxonium.eigenvals(evals_count=2)[1] - fluxonium.eigenvals(evals_count=2)[0]

# charge operator matrix element <0|n|1>
n_mat_elem_fluxonium = fluxonium.matrixelement_table('n_operator', evals_count=5)[0,1]

print("\nFluxonium f01 = ", f01_fluxonium, "GHz")
print("<0|n|1> =", n_mat_elem_fluxonium)

Ratio = (abs(n_mat_elem_fluxonium) / abs(n_mat_elem_transmon))
print("Fluxonium/Transmon charge matrix element ratio squared =", Ratio**2)


Transmon f01 =  3.553307887219426 GHz
<0|n|1> = -0.22326867090985783

Fluxonium f01 =  0.3063882233586539 GHz
<0|n|1> = 0.10190229612356481j
Fluxonium/Transmon charge matrix element ratio squared = 0.2083110776404005


In [8]:
# --- Fluxonium external T1 from EM coupling (scqubits required) ---

import numpy as np
import scqubits as scq

# ---- Given parameters ----
EC_GHz = 0.85          # EC/h (GHz)
EJ_GHz = 5.0           # EJ/h (GHz)
EL_GHz = 1.15          # EL/h (GHz)
Cc_fF  = 0.103         # coupling capacitance to 50-ohm line (fF)
Z0     = 50.0          # ohms

# Choose operating flux bias (units of Phi0)
flux_bias = 0.5

# ---- Constants ----
e  = 1.602176634e-19
h  = 6.62607015e-34
hbar = 1.054571817e-34
pi = np.pi

# ---- Derived: total capacitance from EC ----
#   EC/h = e^2/(2 h CΣ)  ->  CΣ = e^2 / (2 h (EC/h))
Csum_F = e**2 / (2 * h * EC_GHz * 1e9)   # Farad

# ---- Convert caps ----
Cc_F = Cc_fF * 1e-15

# ---- Build fluxonium and get f01 & n01 at chosen bias ----
fluxonium = scq.Fluxonium(
    EJ=EJ_GHz, EC=EC_GHz, EL=EL_GHz,
    flux=flux_bias, cutoff=220, truncated_dim=120
)

# Eigenvalues (GHz) and eigenvectors (NumPy array, columns are eigenvectors)
evals, evecs = fluxonium.eigensys(evals_count=2)   # evals: (2,), evecs: (dim, 2)
f01_GHz = float(evals[1] - evals[0])

psi0 = evecs[:, 0]          # ground state vector
psi1 = evecs[:, 1]          # first excited state vector

# Charge operator matrix (dense NumPy array)
n_op = fluxonium.n_operator()
try:
    n_mat = n_op.full()     # if QuTiP Qobj
except AttributeError:
    n_mat = np.asarray(n_op)

# Matrix element <0|n|1>
n01 = np.vdot(psi0, n_mat @ psi1)   # complex scalar
n01_mag = abs(n01)

# ---- Harmonic-oscillator rescaling pieces ----
wq = 2 * pi * f01_GHz * 1e9                        # rad/s
kappa_HO = Z0 * wq**2 * (Cc_F**2 / Csum_F)         # 1/s

# HO zero-point charge matrix element: n_zpf^HO = sqrt(w / (16 * EC))
EC_rad_s = 2 * pi * EC_GHz * 1e9
nzpf_HO = np.sqrt(wq / (16.0 * EC_rad_s))

# ---- Fluxonium external rate and T1 ----
kappa_flux = kappa_HO * (n01_mag / nzpf_HO)**2     # 1/s
T1_sec = 1.0 / kappa_flux

# ---- Report ----
print(f"Flux bias (Phi0 units): {flux_bias:.3f}")
print(f"CΣ = {Csum_F*1e15:.3f} fF   |   Cc = {Cc_fF:.3f} fF")
print(f"f01 = {f01_GHz:.6f} GHz")
print(f"|n01| (fluxonium) = {n01_mag:.6f}")
print(f"n_zpf^HO          = {nzpf_HO:.6f}")
print(f"kappa_HO   /2π = {kappa_HO/(2*pi):.3e} Hz")
print(f"kappa_flux /2π = {kappa_flux/(2*pi):.3e} Hz")
print(f"T1_ext = {T1_sec*1e6:.3f} µs  ({T1_sec:.3e} s)")


Flux bias (Phi0 units): 0.500
CΣ = 22.789 fF   |   Cc = 0.103 fF
f01 = 0.306388 GHz
|n01| (fluxonium) = 0.101902
n_zpf^HO          = 0.150095
kappa_HO   /2π = 1.373e+01 Hz
kappa_flux /2π = 6.328e+00 Hz
T1_ext = 25149.695 µs  (2.515e-02 s)


In [25]:
#

import numpy as np
import scqubits as scq

# ----- Given (you can tweak) -----
EC_GHz = 0.88          # EC/h in GHz
EJ_GHz = 5.0           # EJ/h in GHz
EL_GHz = 1.15          # EL/h in GHz
Cc_fF  = .22        # coupling capacitor to 50 Ω line, in fF
Z0     = 50.0          # line impedance, Ω
flux_bias = 0.5        # Φext/Φ0

# ----- Constants -----
e  = 1.602176634e-19   # C
h  = 6.62607015e-34    # J*s
pi = np.pi

# Total capacitance from EC/h = e^2/(2 h CΣ)
Csum_F = e**2 / (2 * h * EC_GHz * 1e9)    # F
Cc_F   = Cc_fF * 1e-15                     # F

# Build fluxonium at flux = 0.5 Φ0
qb = scq.Fluxonium(EJ=EJ_GHz, EC=EC_GHz, EL=EL_GHz,
                   flux=flux_bias, cutoff=220, truncated_dim=120)

# f01 (GHz)
evals, _ = qb.eigensys(evals_count=2)
f01_GHz = float(evals[1] - evals[0])

# Your way: n in the energy eigenbasis, then (0,1) element
n01_flux = abs(qb.n_operator(energy_esys=True)[0, 1])   # dimensionless

# External decay of the HO used in EM: κ_HO = Z0 * ω^2 * (Cc^2 / CΣ)
wq = 2 * pi * f01_GHz * 1e9                              # rad/s
kappa_HO = Z0 * wq**2 * (Cc_F**2 / Csum_F)               # 1/s

# HO zero-point charge element: n_zpf^HO = sqrt( w / (16 * EC_rad_s) )
EC_rad_s = 2 * pi * EC_GHz * 1e9
n_zpf_HO = np.sqrt(wq / (16.0 * EC_rad_s))

# Fluxonium external κ and T1
kappa_flux = kappa_HO * (n01_flux / n_zpf_HO)**2         # 1/s
T1_sec = 1.0 / kappa_flux

T1_HO_sec = 1.0 / kappa_HO
print(f"T1_HO (transmon/HO external) : {T1_HO_sec*1e6:.3f} μs  ")

print(f"flux bias           : {flux_bias:.3f} Φ0")
print(f"CΣ, Cc              : {Csum_F*1e15:.3f} fF , {Cc_fF:.3f} fF")
print(f"f01                 : {f01_GHz:.6f} GHz")
print(f"|n01| (fluxonium)   : {n01_flux:.6f}")
print(f"n_zpf^HO            : {n_zpf_HO:.6f}")
print(f"kappa_HO   /2π      : {kappa_HO/(2*pi):.3e} Hz")
print(f"kappa_flux /2π      : {kappa_flux/(2*pi):.3e} Hz")
print(f"T1_ext              : {T1_sec*1e6:.3f} μs  ")


T1_HO (transmon/HO external) : 2119.585 μs  
flux bias           : 0.500 Φ0
CΣ, Cc              : 22.012 fF , 0.220 fF
f01                 : 0.329696 GHz
|n01| (fluxonium)   : 0.105638
n_zpf^HO            : 0.153022
kappa_HO   /2π      : 7.509e+01 Hz
kappa_flux /2π      : 3.578e+01 Hz
T1_ext              : 4447.551 μs  


In [ ]:
# For transmon
import numpy as np

def transmon_T1_external(EC_GHz, Cc_fF, f01_GHz, Z0=50.0):
    """
    External T1 of a (harmonic) transmon coupled via series Cc to a 50-Ω line.
    Uses: kappa_HO = Z0 * w^2 * (Cc^2 / Csum),   T1 = 1/kappa_HO
    Inputs:
        EC_GHz  : charging energy EC/h (GHz)
        Cc_fF   : coupling capacitor (fF)
        f01_GHz : qubit transition frequency (GHz)
        Z0      : line impedance (Ohm), default 50
    Returns:
        dict with Csum_fF, kappa_HO_Hz, T1_HO_sec
    """
    # constants
    e  = 1.602176634e-19
    h  = 6.62607015e-34
    pi = np.pi

    # total capacitance from EC/h = e^2 / (2 h Csum)
    Csum_F = e**2 / (2 * h * EC_GHz * 1e9)      # F
    Cc_F   = Cc_fF * 1e-15                      # F

    # frequency & external rate
    w = 2 * pi * f01_GHz * 1e9                  # rad/s
    kappa = Z0 * w**2 * (Cc_F**2 / Csum_F)      # 1/s

    return {
        "Csum_fF": Csum_F * 1e15,
        "kappa_HO_Hz": kappa / (2 * pi),
        "T1_HO_u_sec": 1.0 / kappa *1e6
    }

# --- example ---
out = transmon_T1_external(EC_GHz=0.165, Cc_fF=0.06, f01_GHz=4.5)
print(out)



{'Csum_fF': 117.39532924035831, 'kappa_HO_Hz': 195.08621503824062, 'T1_HO_u_sec': 815.8184988144751}
